In [1]:
import unibox as ub
import pandas as pd

Using global temporary directory: /tmp/unibox_temp


In [ ]:
df = ub.loads("hf://incantor/user-prompts-vis-entries").to_pandas()

prompts_list = df["prompt"].tolist()
ub.saves(prompts_list, "user_prompts.txt")

In [ ]:
import os
import json
import time
import hashlib
import requests
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
from io import BytesIO
import base64
import threading

# === Config ===
CONTROLLER = "http://127.0.0.1:9000/generate"
CLIENT_ID = "my-client-1"
PROMPT_FILE = "user_prompts.txt"  # optional: use instead of PROMPTS
SAVE_DIR = "outputs"
NUM_WORKERS = 4  # Number of parallel requests
MAX_IN_FLIGHT = NUM_WORKERS * 2  # Allow some buffer

# === Read prompts ===
if os.path.exists(PROMPT_FILE):
    SAVE_DIR = f"outputs_{os.path.splitext(os.path.basename(PROMPT_FILE))[0]}"
    print(f"Using prompts from {PROMPT_FILE} | Saving to {SAVE_DIR}")
    with open(PROMPT_FILE, 'r', encoding='utf-8') as f:
        PROMPTS = [line.strip() for line in f if line.strip()]
else:
    PROMPTS = ["a cat", "a dog", "a castle", "a dragon"] * 4

# === Ensure save dir ===
os.makedirs(SAVE_DIR, exist_ok=True)

# === Save function ===
def save_image_and_metadata(img_b64, metadata, index):
    prefix = f"{index:0{len(str(len(PROMPTS)))}}"
    
    # Compute hash to avoid naming collisions
    meta_str = json.dumps(metadata, sort_keys=True)
    digest = hashlib.sha1(meta_str.encode()).hexdigest()[:8]
    
    img_filename = f"{prefix}_{digest}.png"
    json_filename = f"{prefix}_{digest}.json"

    # Decode and save image
    image = Image.open(BytesIO(base64.b64decode(img_b64.split(',')[-1])))
    image.save(os.path.join(SAVE_DIR, img_filename))

    # Save metadata
    metadata["client_timestamp"] = time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())
    with open(os.path.join(SAVE_DIR, json_filename), "w") as f:
        json.dump(metadata, f, indent=2)

    print(f"Saved: {img_filename}, {json_filename}")

# === Request function with semaphore ===
semaphore = threading.Semaphore(MAX_IN_FLIGHT)

def send(index, prompt, width, height):
    with semaphore:
        try:
            resp = requests.post(
                CONTROLLER,
                json={"prompt": prompt, "width": width, "height": height},
                headers={"X-Client-ID": CLIENT_ID},
                timeout=300
            )
            resp.raise_for_status()
            data = resp.json()
            if "image" in data and "metadata" in data:
                save_image_and_metadata(data["image"], data["metadata"], index)
            else:
                print(f"Failed: Missing image or metadata for prompt: {prompt}")
        except Exception as e:
            print(f"Error at line {index+1} ('{prompt}'): {e}")

# === Main Execution ===
if __name__ == "__main__":
    start = time.time()
    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        futures = [
            executor.submit(send, idx, p, 1024, 1024)
            for idx, p in enumerate(PROMPTS)
        ]
        [f.result() for f in futures]
    print("Done in", round(time.time() - start, 2), "seconds.")

Saved: 02_521016bb.png, 02_521016bb.json
Saved: 03_96f61436.png, 03_96f61436.json
Saved: 01_5da44f13.png, 01_5da44f13.json
Saved: 00_9c6a0286.png, 00_9c6a0286.json
Saved: 05_4cb50416.png, 05_4cb50416.json
Saved: 06_6b8c1704.png, 06_6b8c1704.json
Saved: 04_28581360.png, 04_28581360.json
Saved: 07_b8c00f4b.png, 07_b8c00f4b.json
Saved: 10_c0da91a9.png, 10_c0da91a9.json
Saved: 09_25c812a1.png, 09_25c812a1.json
Saved: 08_33b7d216.png, 08_33b7d216.json
Saved: 11_cffe7f8f.png, 11_cffe7f8f.json
Saved: 13_f1173ae1.png, 13_f1173ae1.json
Saved: 14_fe6ec9aa.png, 14_fe6ec9aa.json
Saved: 12_b15de6a1.png, 12_b15de6a1.json
Saved: 15_4f2090c1.png, 15_4f2090c1.json
Done in 95.22 seconds.


## gradio version:

In [ ]:
import gradio as gr
import requests
import base64
import json
from PIL import Image
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor

CONTROLLER = "http://127.0.0.1:9000/generate"
CLIENT_ID = "my-client-1"
NUM_WORKERS = 4


def decode_image(img_b64):
    image = Image.open(BytesIO(base64.b64decode(img_b64.split(',')[-1])))
    return image


def generate_batch(prompt_text, width, height, front, back):
    prompts_raw = [line.strip() for line in prompt_text.strip().splitlines() if line.strip()]
    prompts = [f"{front}, {p}, {back}".strip(", ") for p in prompts_raw]
    results = [None] * len(prompts)

    def send(index, prompt):
        try:
            resp = requests.post(
                CONTROLLER,
                json={"prompt": prompt, "width": width, "height": height},
                headers={"X-Client-ID": CLIENT_ID},
                timeout=300,
            )
            resp.raise_for_status()
            data = resp.json()
            if "image" in data and "metadata" in data:
                image = decode_image(data["image"])
                json_str = json.dumps(data["metadata"], indent=2)
                results[index] = (image, json_str)
            else:
                results[index] = (None, f"Missing fields for prompt: {prompt}")
        except Exception as e:
            results[index] = (None, f"Error for prompt '{prompt}': {str(e)}")

    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        futures = [executor.submit(send, idx, p) for idx, p in enumerate(prompts)]
        [f.result() for f in futures]

    images = [r[0] for r in results if r[0] is not None]
    metadata = [r[1] for r in results if r[0] is not None]
    errors = [r[1] for r in results if r[0] is None]

    return images, metadata, "\n".join(errors)


with gr.Blocks() as demo:
    gr.Markdown("""# Batch Image Generator\nEnter one prompt per line.""")
    with gr.Row():
        prompt_input = gr.Textbox(lines=10, label="Prompts (one per line)")
        with gr.Column():
            text_front = gr.Textbox(label="Text Front")
            text_back = gr.Textbox(label="Text Back")
            width = gr.Number(value=1024, label="Width")
            height = gr.Number(value=1024, label="Height")
            run_button = gr.Button("Generate")

    gallery = gr.Gallery(label="Generated Images", columns=4)
    metadata_output = gr.JSON(label="Metadata")
    error_output = gr.Textbox(label="Errors", lines=4)

    run_button.click(
        fn=generate_batch,
        inputs=[prompt_input, width, height, text_front, text_back],
        outputs=[gallery, metadata_output, error_output]
    )

if __name__ == "__main__":
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861


* Running on public URL: https://ccfcff33c1fafc5a5e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
